In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import tensorflow.lite as tflite
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.applications.xception import decode_predictions
from tensorflow.keras.preprocessing.image import ImageDataGenerator

C:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
train_path = './data/train'
test_path = './data/test'

In [3]:
model = keras.models.load_model('bees-wasps.h5')


In [4]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()

with open('bees-wasps.tflite', 'wb') as f_out:
    f_out.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\touit\AppData\Local\Temp\tmphgvp8ti8\assets


INFO:tensorflow:Assets written to: C:\Users\touit\AppData\Local\Temp\tmphgvp8ti8\assets


In [5]:
import os
os.path.getsize('./bees-wasps.tflite')

44866200

In [6]:
#Q1 = 44866200

In [7]:
interpreter = tflite.Interpreter(model_path='bees-wasps.tflite')
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

In [8]:
interpreter.get_tensor_details()

[{'name': 'serving_default_conv2d_input:0',
  'index': 0,
  'shape': array([  1, 150, 150,   3]),
  'shape_signature': array([ -1, 150, 150,   3]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}},
 {'name': 'sequential/conv2d/BiasAdd/ReadVariableOp',
  'index': 1,
  'shape': array([32]),
  'shape_signature': array([32]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}},
 {'name': 'sequential/conv2d/Conv2D',
  'index': 2,
  'shape': array([32,  3,  3,  3]),
  'shape_signature': array([32,  3,  3,  3]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], 

In [9]:
interpreter.get_output_details()[0]

{'name': 'StatefulPartitionedCall:0',
 'index': 13,
 'shape': array([1, 1]),
 'shape_signature': array([-1,  1]),
 'dtype': numpy.float32,
 'quantization': (0.0, 0),
 'quantization_parameters': {'scales': array([], dtype=float32),
  'zero_points': array([], dtype=int32),
  'quantized_dimension': 0},
 'sparsity_parameters': {}}

In [10]:
input_index

0

In [11]:
output_index

13

In [12]:
#Q2 13

In [13]:
from io import BytesIO
from urllib import request
from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img


def preprocess_input(x):
    x /= 255
    x -= 1.
    return x

In [24]:
img_dl = download_image('https://habrastorage.org/webt/rt/d9/dh/rtd9dhsmhwrdezeldzoqgijdg8a.jpeg')

In [25]:
img_prep = prepare_image(img_dl,(150,150))

C:\Users\touit\AppData\Local\Temp\ipykernel_22308\3119706847.py:17: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  img = img.resize(target_size, Image.NEAREST)


In [26]:
x = np.array(img_prep, dtype='float32')
X = np.array([x])

X = preprocess_input(X)

In [27]:
X.shape

(1, 150, 150, 3)

In [28]:
X

array([[[[ 0.8901961 ,  0.8117647 ,  0.7176471 ],
         [ 0.8745098 ,  0.84313726,  0.9529412 ],
         [ 0.827451  ,  0.79607844,  0.9137255 ],
         ...,
         [-0.41960782, -0.3333333 , -0.67058825],
         [-0.30196077, -0.18431371, -0.69411767],
         [-0.40392154, -0.27843136, -0.7647059 ]],

        [[ 0.8980392 ,  0.81960785,  0.7411765 ],
         [ 0.8352941 ,  0.81960785,  0.92156863],
         [ 0.8039216 ,  0.7882353 ,  0.8980392 ],
         ...,
         [-0.45098037, -0.32549018, -0.6784314 ],
         [-0.05882353,  0.01176476, -0.62352943],
         [-0.09803921, -0.01176471, -0.6313726 ]],

        [[ 0.85882354,  0.7647059 ,  0.6392157 ],
         [ 0.827451  ,  0.81960785,  0.92941177],
         [ 0.8117647 ,  0.7882353 ,  0.92156863],
         ...,
         [-0.35686272, -0.24705881, -0.64705884],
         [ 0.01176476,  0.05098045, -0.5921569 ],
         [-0.1372549 , -0.06666666, -0.5921569 ]],

        ...,

        [[-0.9372549 , -0.8666667 , -0

In [29]:
#Q3 = 0.7450980 ou 0.8901961 (150)

In [30]:
interpreter.set_tensor(input_index,X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)

In [33]:
preds

array([[0.1389202]], dtype=float32)

In [ ]:
#Q4 array([[0.1389202]], dtype=float32)

In [ ]:
#Docker Q5 662mb

In [ ]:
train_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_ds = train_gen.flow_from_directory(
    train_path,
    target_size=(150, 150),
    batch_size=32
)

In [ ]:
val_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

val_ds = val_gen.flow_from_directory(
    test_path,
    target_size=(150, 150),
    batch_size=32,
    shuffle=False
)

In [ ]:
inputs = keras.Input(shape=(150,150,3))

first_layer_conv2D = keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu')(inputs)

first_layer_pooling = keras.layers.MaxPooling2D(pool_size=(2,2))(first_layer_conv2D)

first_layer_flatten = keras.layers.Flatten()(first_layer_pooling)

inner_layer = keras.layers.Dense(64,activation='relu')(first_layer_flatten)

outputs = keras.layers.Dense(1, activation='sigmoid')(inner_layer)

model = keras.Model(inputs, outputs)


optimizer = keras.optimizers.SGD(lr=0.002, momentum=0.8)
loss = keras.losses.BinaryCrossentropy()

model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])


In [ ]:
#Q1 
# binary crossentropy

In [ ]:
#Q2

In [ ]:
model.summary()

In [ ]:
# 11,215,873

In [ ]:
#Q3

In [ ]:
img_generator = ImageDataGenerator(rescale=1./255)

train_generator = img_generator.flow_from_directory(
    train_path,
    target_size=(150, 150),
    batch_size=20,
    shuffle=True,
    class_mode='binary'
)

test_generator = img_generator.flow_from_directory(
    test_path,
    target_size=(150, 150),
    batch_size=20,
    shuffle=True,
    class_mode='binary'

)

history = model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator
)

In [ ]:
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='val')
plt.xticks(np.arange(10))
plt.legend()

In [ ]:
np.median(history.history['accuracy'])

In [ ]:
# 0.7647538781166077

In [ ]:
#Q4

In [ ]:
np.std(history.history['loss'])

In [ ]:
#0.09010267966314418

In [ ]:
#Q5

In [ ]:
img_generator = ImageDataGenerator(
    rescale=1./255,
    rotation_range=50,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = img_generator.flow_from_directory(
    train_path,
    target_size=(150, 150),
    batch_size=20,
    shuffle=True,
    class_mode='binary'
)

test_generator = img_generator.flow_from_directory(
    test_path,
    target_size=(150, 150),
    batch_size=20,
    shuffle=True,
    class_mode='binary'

)

history = model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator
)

In [ ]:
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='val')
plt.xticks(np.arange(10))
plt.legend()

In [ ]:
np.mean(history.history['loss'])

In [ ]:
# 0.473125821352005

In [ ]:
#Q6 What's the average of test accuracy for the last 5 epochs (from 6 to 10) for the model trained with augmentations?

In [ ]:
np.mean(history.history['val_accuracy'][5:])

In [ ]:
# 0.7779956340789795
